In [ ]:
!pip install torch torchvision numpy matplotlib Pillow requests

In [ ]:
# === Import Necessary Libraries ===
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
from PIL import Image
import requests
import json
import os
import copy # To ensure model 2 is a separate instance
import matplotlib.pyplot as plt # Import for displaying images

# === Configuration ===
# Use GPU if available in Colab, otherwise CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"--- Using device: {DEVICE} ---")

# Epsilon: Controls the magnitude of the noise added.
EPSILON = 0.02 # You can experiment with this value (e.g., 0.01, 0.05)

# ImageNet class labels (fetch if not present)
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
LABELS_PATH = "/content/imagenet-simple-labels.json" # Standard Colab path

# Royalty-Free Sample image URL (Labrador from PyTorch Hub repo)
IMAGE_URL = "https://raw.githubusercontent.com/pytorch/hub/master/images/dog.jpg"
# Define explicit paths for saving in Colab's temporary storage
ORIGINAL_IMAGE_SAVE_PATH = "/content/original_image.png"
NOISE_IMAGE_SAVE_PATH = "/content/noise_visualization.png"
ADVERSARIAL_IMAGE_SAVE_PATH = "/content/adversarial_image.png"

# === Helper Functions ===

def get_imagenet_labels():
    """Downloads or loads ImageNet labels."""
    if not os.path.exists(LABELS_PATH):
        print("Downloading ImageNet labels...")
        response = requests.get(LABELS_URL)
        response.raise_for_status()
        with open(LABELS_PATH, 'w') as f:
            f.write(response.text)
        print(f"Labels saved to {LABELS_PATH}")
    with open(LABELS_PATH) as f:
        labels = json.load(f)
    return labels

def download_and_load_image(url, save_path):
    """Downloads an image, saves it, and loads it as PIL."""
    if not os.path.exists(save_path):
        print(f"Downloading image from {url}...")
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Original image downloaded and saved to {save_path}")
    else:
        print(f"Original image {save_path} already exists.")
    # Load the saved image using PIL
    img_pil = Image.open(save_path).convert('RGB')
    return img_pil

def preprocess_pil_image(img_pil):
    """Preprocesses a PIL image for ResNet."""
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img_t = preprocess(img_pil)
    batch_t = torch.unsqueeze(img_t, 0) # Add batch dimension
    return batch_t.to(DEVICE)

def deprocess_tensor_to_pil(tensor):
    """Converts a normalized tensor back to a PIL Image."""
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    tensor = tensor.squeeze(0).cpu().detach().numpy()
    tensor = tensor.transpose(1, 2, 0)
    tensor = std * tensor + mean
    tensor = np.clip(tensor, 0, 1)
    # Convert float [0,1] to uint8 [0,255]
    img_pil = Image.fromarray((tensor * 255).astype(np.uint8))
    return img_pil

def save_pil_image(img_pil, filename):
    """Saves a PIL image object to a file."""
    try:
        img_pil.save(filename)
        print(f"Image successfully saved to {filename}")
    except Exception as e:
        print(f"Error saving image {filename}: {e}")


def fgsm_attack(model, loss_fn, image, epsilon, target_label):
    """Performs the Fast Gradient Sign Method attack."""
    image.requires_grad = True
    output = model(image)
    loss = loss_fn(output, target_label)
    model.zero_grad()
    loss.backward()
    gradient = image.grad.data
    sign_gradient = gradient.sign()
    perturbed_image = image + epsilon * sign_gradient
    perturbed_image = perturbed_image.detach()
    image.requires_grad = False # Clean up
    return perturbed_image, sign_gradient

# === Main Execution ===

print("\n--- Starting Adversarial Attack Experiment ---")

# 1. Setup: Load labels and download/save/load sample image
imagenet_labels = get_imagenet_labels()
# Download if needed, and load the original PIL image
original_pil_image = download_and_load_image(IMAGE_URL, ORIGINAL_IMAGE_SAVE_PATH)
# Save the original image again (or confirm it was saved) - Redundant if download_and_load_image saves it.
# save_pil_image(original_pil_image, ORIGINAL_IMAGE_SAVE_PATH) # Already saved in helper


# 2. Load the first pre-trained model (Attacker's view / Generator)
print("\n--- Loading Model 1 (for attack generation) ---")
model1 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model1.eval()
model1.to(DEVICE)

# 3. Preprocess the original image for Model 1
original_image_tensor = preprocess_pil_image(original_pil_image)

# 4. Get the original prediction using Model 1
print("\n--- Classifying Original Image with Model 1 ---")
with torch.no_grad():
    output1_orig = model1(original_image_tensor)

original_prob = torch.nn.functional.softmax(output1_orig, dim=1)
original_pred_prob, original_pred_idx = torch.max(original_prob, 1)
original_pred_label = imagenet_labels[original_pred_idx.item()]

print(f"Model 1 Original Prediction: '{original_pred_label}' (Index: {original_pred_idx.item()})")
print(f"Confidence: {original_pred_prob.item():.4f}")

# 5. Generate the adversarial example using FGSM
print(f"\n--- Generating Adversarial Example (FGSM, Epsilon={EPSILON}) ---")
loss_function = nn.CrossEntropyLoss()
target = original_pred_idx

adversarial_image_tensor, noise_sign_tensor = fgsm_attack(
    model1, loss_function, original_image_tensor.clone(), EPSILON, target
)

# 6. Prepare Adversarial Image & Noise Visualization for saving/display
print("\n--- Preparing & Saving Adversarial Results ---")
# Adversarial Image (PIL format)
adversarial_pil_image = deprocess_tensor_to_pil(adversarial_image_tensor)
save_pil_image(adversarial_pil_image, ADVERSARIAL_IMAGE_SAVE_PATH)

# Noise Visualization (PIL format)
noise_vis_tensor = noise_sign_tensor.squeeze(0).cpu()
noise_vis_tensor = (noise_vis_tensor + 1) / 2 # Scale sign {-1, 1} to {0, 1}
noise_pil_image = transforms.ToPILImage()(noise_vis_tensor)
save_pil_image(noise_pil_image, NOISE_IMAGE_SAVE_PATH)

# 7. Load the second pre-trained model (Victim's view / Classifier)
print("\n--- Loading Model 2 (for classifying adversarial image) ---")
model2 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model2.eval()
model2.to(DEVICE)

# 8. Load the *saved* adversarial image and preprocess it for Model 2
print(f"\n--- Loading Adversarial Image from {ADVERSARIAL_IMAGE_SAVE_PATH} ---")
# We use the already generated 'adversarial_pil_image' for consistency before display
# but simulate the loading process for classification tensor
adversarial_pil_loaded = Image.open(ADVERSARIAL_IMAGE_SAVE_PATH).convert('RGB')
adversarial_image_loaded_tensor = preprocess_pil_image(adversarial_pil_loaded)

# 9. Classify the adversarial image using Model 2
print("\n--- Classifying Adversarial Image with Model 2 ---")
with torch.no_grad():
    output2_adv = model2(adversarial_image_loaded_tensor)

adv_prob = torch.nn.functional.softmax(output2_adv, dim=1)
adv_pred_prob, adv_pred_idx = torch.max(adv_prob, 1)
adv_pred_label = imagenet_labels[adv_pred_idx.item()]

print(f"Model 2 Adversarial Prediction: '{adv_pred_label}' (Index: {adv_pred_idx.item()})")
print(f"Confidence: {adv_pred_prob.item():.4f}")
if original_pred_idx.item() != adv_pred_idx.item():
    print(">>> Attack Successful: Model 2 misclassified the adversarial image! <<<")
else:
    print(">>> Attack Unsuccessful: Model 2 correctly classified the adversarial image. (Try increasing Epsilon?) <<<")


# 10. Display Images in Colab Output
print("\n--- Displaying Images ---")

plt.figure(figsize=(18, 6)) # Adjusted figure size for better layout

# Subplot 1: Original Image
plt.subplot(1, 3, 1)
# Convert PIL to NumPy array for imshow
plt.imshow(np.array(original_pil_image))
plt.title(f"Original Image\nPred (M1): '{original_pred_label}'\n({original_pred_prob.item()*100:.2f}%)", fontsize=10)
plt.axis('off')

# Subplot 2: Noise Visualization
plt.subplot(1, 3, 2)
# Convert PIL to NumPy array for imshow
plt.imshow(np.array(noise_pil_image))
plt.title(f"Noise Visualization\n(FGSM Sign Gradient, eps={EPSILON})", fontsize=10)
plt.axis('off')

# Subplot 3: Adversarial Image
plt.subplot(1, 3, 3)
# Convert PIL to NumPy array for imshow
plt.imshow(np.array(adversarial_pil_image))
plt.title(f"Adversarial Image\nPred (M2): '{adv_pred_label}'\n({adv_pred_prob.item()*100:.2f}%)", fontsize=10)
plt.axis('off')

plt.tight_layout() # Adjust layout to prevent overlap
plt.show() # Display the plots inline


# 11. Final Summary in Terminal
print("\n--- Experiment Summary ---")
print(f"Original Image Prediction (Model 1): '{original_pred_label}' ({original_pred_prob.item():.4f})")
print(f"Adversarial Image Prediction (Model 2): '{adv_pred_label}' ({adv_pred_prob.item():.4f})")
print("-" * 20)
print("Files saved in Colab environment:")
print(f"  Original Image:       {ORIGINAL_IMAGE_SAVE_PATH}")
print(f"  Noise Visualization:  {NOISE_IMAGE_SAVE_PATH}")
print(f"  Adversarial Image:    {ADVERSARIAL_IMAGE_SAVE_PATH}")
print("-" * 20)
print("You can also view/download these files from the file browser panel on the left in Colab.")
print("\n--- Tutorial Finished ---")